In [1]:
%pip install opencv-python-headless pyyaml
%pip install numpy opencv-python-headless

     ---------------------------------------- 38.5/38.5 MB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
#load Yaml
with open(r'C:\Users\alist\OneDrive\Desktop\YOLO project\1_datapreparation\data.yaml', mode ='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)

['person', 'car', 'chair', 'bottle', 'bird', 'sofa', 'cycle', 'horse', 'bus']


In [3]:
#load Yolo Model 
yolo = cv2.dnn.readNetFromONNX(r'C:\Users\alist\OneDrive\Desktop\YOLO project\prediction\Model13\weights\best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
#open video file for bject detecttion
video_path = r'C:\Users\alist\OneDrive\Desktop\YOLO project\prediction\video.mp4'
cap = cv2.VideoCapture(video_path)

In [10]:
while True:
    ret, frame = cap.read()
    
    if not ret: # if no frame is grabbed (end of the video), break out
        break
    #preprocess frame and performs YOLO object
    max_rc = max(frame.shape[:2])
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.unit8)
    input_image[0:frame.shape[0],0:frame.shape[1]] = frame
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO),swapRB =True, crop= False)
    yolo.setINput(blob)
    preds = yolo.forward()
    
    #process detection and draw boxes.
    detection = preds[0] #extract detection for predictions
    boxes = []    #list to store bounding boxes
    confidences = []   #list to store confidences
    classes = []   #list to store indices
    
    image_w, image_h = input_image.shape[:2] #get width and height
    x_factor = image_w / INPUT_WH_YOLO
    y_factor = image_h/ INPUT_WH_YOLO  #scaling factor for h
    
    for i in range(len(detection)): #Iterate through detection
        row = detection[i]   #extract 
        confidences = row[4]
        if confidences > 0.4:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx,cy,w,h = row[0:4]
                left = int((cx - 0.5 * w) * x_factor)
                top = int((cy - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top,left+ width, top+height])
                
                confidences.append(class_score)
                boxes.append(box)
                classes.append(class_id)
    boxes_np = np.array(boxes)
    confidences_np = np.array()
    
    #perform non-maximum suppression to remove redundant(duplicate) bounding boxes
    output = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45)
    if len(output) >0 :
        index = output.flatten()
    else:
        index = np.empty((0,), dtype=int)
        
    #draw bounding boxes and labels on the frame
    for ind in index:
        x , y, w, h = boxes_np[ind]
        bb_conf = int(confidences_np[ind] * 100)
        class_id = classes[ind]
        class_name = labels[class_id]
        
        text = f'{class_name}:{bb_conf}%' #create label text
        cv2.rectangle(frame, (x, y), (x + w, y+h), (0, 255,0),2)
        cv2.rectangle(frame,(x,y -30),(x+w,y),(255,255,255), -1)
        cv2.putText(frame, text, (x,y -10), cv2.FONT_HERSHEY_PLAIN, 0.7,(0,0,0),1)
        
    cv2.imshow('YOLO Object Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()


In [ ]:
#preprocess frame and performs YOLO object
